# Creating a Parsl example 

**NOTE:** This example requires the parsl package in your cluster environment. Follow [this link](https://docs.virtualclusters.org/userguide/gettingstarted/#9-environments) to learn more about defining environments in your virtual cluster . 

The following example follows the [Parsl introduction example](https://parsl.readthedocs.io/en/latest/parsl-introduction.html), but using a VC3 controller for the cluster:
For full documentation, visit: https://parsl.readthedocs.io

## Configuring the cluster
First, we will configure a local condor cluster through the IPyParallel engine, restricting TCP ports to 9100 to 9800 for our node:


In [7]:
import parsl
from parsl.app.app import python_app, bash_app

from parsl.executors.ipp_controller import Controller
from parsl.config import Config
from parsl.executors.ipp import IPyParallelExecutor
from vc3controller import VC3Controller
from requests import get

config = Config(
    executors=[
        IPyParallelExecutor(
            label='vc3_local_ipp',
            provider=parsl.providers.CondorProvider(
                channel=parsl.channels.LocalChannel(),
                nodes_per_block=1,
                init_blocks=1,
                max_blocks=1,
                requirements='',
                walltime="01:00:00",
            ),
            engine_debug_level='DEBUG',
            controller=VC3Controller(public_ip='{0}'.format(get('https://api.ipify.org').text), port_range='9100,9800')
        )
    ],
)


Now, we can load the parsl configuration and define a python application

In [12]:
parsl.load(config)

@python_app
def hello ():
    import socket
    return 'Hello from hostname: {0}'.format(socket.gethostname())

Next, we ask for the result and wait.

In [9]:
print(hello().result())

Hello from hostname: qa-xp-001.crc.nd.edu
